In [2]:
from dotenv import load_dotenv
import os

load_dotenv(".env")
display(os.environ.get("NEO4J_USERNAME"))

'neo4j'

In [3]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable

In [4]:
interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview",
)
output = interpreter_assistant.invoke({"content": "What's 10 - 4 raised to the 2.7"})
output

[ThreadMessage(id='msg_8aIa2GxuXucGmkCw1znvcUkL', assistant_id='asst_WY0p3kxr7hbMkM7SltOjwvm8', content=[MessageContentText(text=Text(annotations=[], value='The calculation of \\(10 - 4^{2.7}\\) yields approximately -32.224.'), type='text')], created_at=1704655818, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_GvePkCfnsWKh1RPsEQ1NYqfy', thread_id='thread_SlObpZXmR2Gvh1g79OqcIPHW')]

In [5]:
from langchain.tools import DuckDuckGoSearchRun, E2BDataAnalysisTool
tools = [E2BDataAnalysisTool(), DuckDuckGoSearchRun()]

In [6]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions. You can also search the internet.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True,
)

In [7]:
%config Jupyter.display.width=1000

In [8]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke({"content": "What's the weather in SF today divided by 2.7"})

{'content': "What's the weather in SF today divided by 2.7",
 'output': "The current temperature in San Francisco is 50°F. To find out what the weather in San Francisco today is divided by 2.7, we simply divide 50 by 2.7. Let's do that calculation.\nThe result of 50°F divided by 2.7 is approximately 18.52.",
 'thread_id': 'thread_qIiSfLDBEF5jGy6ahIkl5Cg1',
 'run_id': 'run_URjJIn1ShSuf1Q7mHWLEPJP9'}

In [9]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True,
)

In [10]:
from langchain_core.agents import AgentFinish


def execute_agent(agent, tools, input):
    tool_map = {tool.name: tool for tool in tools}
    response = agent.invoke(input)
    while not isinstance(response, AgentFinish):
        tool_outputs = []
        for action in response:
            tool_output = tool_map[action.tool].invoke(action.tool_input)
            print(action.tool, action.tool_input, tool_output, end="\n\n")
            tool_outputs.append(
                {"output": tool_output, "tool_call_id": action.tool_call_id}
            )
        response = agent.invoke(
            {
                "tool_outputs": tool_outputs,
                "run_id": action.run_id,
                "thread_id": action.thread_id,
            }
        )

    return response

In [11]:
response = execute_agent(agent, tools, {"content": "What's 10 - 4 raised to the 2.7"})
print(response.return_values["output"])

e2b_data_analysis {'python_code': 'result = 10 - 4 ** 2.7\nprint(result)'} {"stdout": "-32.22425314473263", "stderr": "", "artifacts": []}

\( 10 - 4^{2.7} \) is approximately \( -32.224 \).


In [12]:
next_response = execute_agent(
    agent,
    tools,
    {"content": "now add 17.241", "thread_id": response.return_values["thread_id"]},
)
print(next_response.return_values["output"])

e2b_data_analysis {'python_code': 'result = 10 - 4 ** 2.7 + 17.241\nprint(result)'} {"stdout": "-14.983253144732629", "stderr": "", "artifacts": []}

When you add \( 17.241 \) to the previous result, you get approximately \( -14.983 \).


In [ ]:
agent = OpenAIAssistantRunnable(assistant_id="<ASSISTANT_ID>", as_agent=True)